In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mne
import mne

     |████████████████████████████████| 7.4 MB 35.0 MB/s 


In [ ]:
import numpy as np
import scipy.io

In [ ]:
"""
cnt: the continuous EEG signals, size [time x channels]. The array is stored in datatype INT16. To convert it to uV values, use cnt= 0.1*double(cnt); in Matlab.
mrk: structure of target cue information with fields (the file of evaluation data does not contain this variable)
pos: vector of positions of the cue in the EEG signals given in unit sample, length #cues
y: vector of target classes (-1 for class one or 1 for class two), length #cues
nfo: structure providing additional information with fields
fs: sampling rate,
clab: cell array of channel labels,
classes: cell array of the names of the motor imagery classes,
xpos: x-position of electrodes in a 2d-projection,
ypos: y-position of electrodes in a 2d-projection.
"""

In [ ]:
from sklearn.utils import shuffle
%cd '/content/drive/MyDrive/EEG project/BCIdataset'

file_name = 'BCICIV_calib_ds1'
name_list = []
for i in ['b','c','d','g']:
  name = file_name + i + '.mat'
  name_list.append(name)

BCI_data_train = []
BCI_label_train = []
BCI_data_test = []
BCI_label_test = []

for i in name_list:
  print(i)
  mat = scipy.io.loadmat(i) 
  sfreq = mat['nfo']['fs'][0][0][0][0]
  EEG = mat['cnt'].T
  nchannels, nsamples = EEG.shape
  chan_names = [s[0] for s in mat['nfo']['clab'][0][0][0]]
  event_onsets  = mat['mrk'][0][0][0]
  event_codes   = mat['mrk'][0][0][1]

  labels = np.zeros((1, nsamples), int)
  labels[0, event_onsets] = event_codes

  cl_lab = [s[0] for s in mat['nfo']['classes'][0][0][0]]
  cl1    = cl_lab[0]
  cl2    = cl_lab[1]

  chan_list = [chan_names.index(i) for i in ['Cz','C3','C4']]

  # store the trials
  trials = {}

  # time window to extract for trial
  window = np.arange(int(2*sfreq), int(6*sfreq))

  nsamples = len(window)

  for cl, code in zip(cl_lab, np.unique(event_codes)):  
      # Extract the onsets for the class
      cl_onsets = event_onsets[event_codes == code]   
      # Allocate memory for the trials
      trials[cl] = np.zeros((nchannels, nsamples, len(cl_onsets))) 
      # Extract each trial
      for i, onset in enumerate(cl_onsets):
          trials[cl][:,:,i] = EEG[:, window+onset]

  left_hand  = np.rollaxis(trials[cl1], 2, 0)  
  right_hand = np.rollaxis(trials[cl2], 2, 0)  

  data = np.concatenate([left_hand, right_hand])
  print(len(data))
  # create labels 
  Y = np.concatenate([-np.ones(left_hand.shape[0]),
            np.ones(right_hand.shape[0])])
  
  data,Y = shuffle(data, Y,)

  for i in data[:160]:
    v = []
    for j in chan_list:
      v.append(i[j])
    v = np.array(v)
    v = v.T
    v = v.reshape((20,20,3))
    BCI_data_train.append(v)

  BCI_label_train.extend(Y[:160])

  for i in data[160:]:
    v = []
    for j in chan_list:
      v.append(i[j])
    v = np.array(v)
    v = v.T
    v = v.reshape((20,20,3))
    BCI_data_test.append(v)
  
  BCI_label_test.extend(Y[160:])


/content/drive/MyDrive/EEG project/BCIdataset
BCICIV_calib_ds1b.mat
200
BCICIV_calib_ds1c.mat
200
BCICIV_calib_ds1d.mat
200
BCICIV_calib_ds1g.mat
200


In [ ]:
BCI_data_train = np.array(BCI_data_train)
BCI_label_train = np.array(BCI_label_train)
BCI_data_test = np.array(BCI_data_test)
BCI_label_test = np.array(BCI_label_test)
print(BCI_data_train.shape)
print(BCI_label_train.shape)

(640, 20, 20, 3)
(640,)


In [ ]:
%cd '/content/drive/MyDrive/EEG project'
def normalization(data):
  _range = np.max(data) - np.min(data)
  return (data - np.min(data)) / _range
def preprocess(data):
  norm_data = normalization(data)
  rgb_data = norm_data*255
  rgb_data = rgb_data.astype(np.uint8)
  return rgb_data

rgb_data_train = preprocess(BCI_data_train)
rgb_data_test = preprocess(BCI_data_test)

np.save(file="bci_rgb_eeg_train.npy", arr=rgb_data_train)
np.save(file="bci_labels_train.npy", arr=BCI_label_train)

np.save(file="bci_rgb_eeg_test.npy", arr=rgb_data_test)
np.save(file="bci_labels_test.npy", arr=BCI_label_test)

/content/drive/MyDrive/EEG project


In [ ]:
chan_list = [chan_names.index(i) for i in ['Cz','C3','C4']]
chan_list

[28, 26, 30]

In [ ]:
data.shape

(200, 59, 400)

In [ ]:
Y

array([-1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,
        1., -1., -1., -1.,  1., -1., -1.,  1.,  1., -1.,  1., -1.,  1.,
        1.,  1.,  1., -1., -1., -1.,  1.,  1., -1.,  1.,  1., -1., -1.,
        1.,  1., -1., -1.,  1.,  1., -1., -1., -1.,  1.,  1., -1.,  1.,
        1., -1.,  1., -1., -1.,  1., -1., -1.,  1., -1.,  1., -1.,  1.,
       -1., -1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1., -1.,  1., -1.,
       -1.,  1.,  1., -1.,  1.,  1.,  1., -1., -1., -1.,  1.,  1., -1.,
       -1.,  1., -1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1., -1., -1.,
       -1.,  1., -1., -1.,  1., -1., -1., -1., -1., -1., -1.,  1., -1.,
        1.,  1., -1., -1., -1.,  1., -1.,  1., -1., -1.,  1.,  1.,  1.,
       -1., -1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1., -1.,
       -1., -1., -1.,  1.,  1., -1.,  1.,  1.,  1., -1., -1.,  1., -1.,
        1.,  1., -1., -1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1.,
        1., -1., -1., -1., -1.,  1.,  1.,  1., -1., -1., -1.,  1